<a href="https://colab.research.google.com/github/lukmandev/NBC-Twitter/blob/master/Naive_Bayes_Text_Classification_Tweet_Prostitute_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pada notebook ini menampilkan metode klasifikasi dokumen teks menggunakan metode Naive Bayes untuk melakukan prediksi data twitter yang termasuk dalam kategori prostitusi atau bukan prostitusi.

Dataset diambil dari twitter sebanyak 40000 data termasuk dalam kategori true (prostitusi) sebanyak 20000 dan false (bukan prostitusi) sebanyak 20000 data.


Pertama, import modul python yang digunakan untuk proses preprocessing dan pembuatan model untuk dilakukan klasifikasi.

In [0]:
import pandas as pd
import string
import numpy as np
import re
import random
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB

Import semua dataset yang akan dilakukan klasifikasi


In [0]:
data_train_true = pd.read_excel('/content/twitter-prostitute.xlsx')
data_train_false = pd.read_excel('/content/twitter-not-prostitute.xlsx')

Gabungkan data true (prostitute) dan data false (not prostitute)

In [0]:
dataset = pd.concat([data_train_true, data_train_false], ignore_index = True)

In [4]:
print(dataset[:5])

                    id  ... status
0  1255052854739398658  ...      1
1  1255052799798202373  ...      1
2  1255052613646573569  ...      1
3  1255052558667661312  ...      1
4  1255052557061287938  ...      1

[5 rows x 5 columns]


In [5]:
print(dataset['tweet'][:5])

0    AvaiL BO yaa beb😙\nWA 0831 9315 9762\n#AvailJo...
1    Include exclude Ready beb \nWa 0831 9315 9762\...
2    AvaiL Jogja Minat DM aja😍\nFasht Respon.\n#Ava...
3    MAEN SANTAI GA BURU" \nFULL SERVICE NO ANAL US...
4    New bie...Ready ya..2 slot aja 085647266101\n#...
Name: tweet, dtype: object


In [6]:
print(len(dataset))

40000


Setelah data berhasil dibuka, saatnya melakukan preprocessing pada teks. Yang akan dilakukan preprocessing yaitu data pada kolom **tweet** dimana data ini berisi tweet yang diambil dari twitter.

Ada beberapa process preprocessing yaitu cleaning text atau membersihkan text dari noise dan tokenizing yaitu memecah semua text menjadi per kata.

In [0]:
stopwords_file = open("/content/stopwords-id.txt", 'r')
stopwords = [x.strip() for x in stopwords_file.readlines()]
stopwords.extend(['by', 'rt', 'via'])

def cleaning(text):
	text = re.sub(r'<[^>]+>', '', text) #delete html tags
	text = re.sub(r'\S*twitter.com\S*', '', text)   #delete twitter image
	text = re.sub(r'https?://[A-Za-z0-9./]+','',text) #delete url
	text = re.sub(r'@[A-Za-z0-9]+','',text) #delete user mention
	text = re.sub(r'#[A-Za-z0-9]+','',text) #delete twitter hashtag
	text = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)','', text) #delete number
	text = re.sub(r"[^a-zA-Z]", " ", text) #only accept alphabet char
	text = re.sub(r"(\w)(\1{2,})", r'\1', text) #delete repeated char
	text = re.sub(r"\b[a-zA-Z]\b", "", text) #remove single character
	text = text.lower() #change to lowercase
	return text

def tokenize(text):
	#disini diisi dengan stop words
	words = text.split();
	words = [w for w in words if w not in stopwords]
	return words

Cleaning text pada data tweet

In [0]:
dataset['tweet'] = dataset.tweet.map(lambda x: cleaning(x))

In [9]:
print(dataset['tweet'][:5])

0             avail bo yaa beb  wa                    
1    include exclude ready beb  wa                 ...
2    avail jogja minat dm aja  fasht respon        ...
3    maen santai ga buru   full service no anal use...
4      new bie   ready ya   slot aja                  
Name: tweet, dtype: object


Tokenizing text pada data tweet

In [0]:
dataset['tweet'] = dataset.tweet.apply(lambda x: tokenize(x))

In [11]:
print(dataset['tweet'][:5])

0                            [avail, bo, yaa, beb, wa]
1                   [include, exclude, ready, beb, wa]
2        [avail, jogja, minat, dm, aja, fasht, respon]
3    [maen, santai, ga, buru, full, service, no, an...
4                     [new, bie, ready, ya, slot, aja]
Name: tweet, dtype: object


Menggabungkan semua tweet menggunakan spasi untuk dilakukan tahap selanjutnya yaitu pembuatan vektor

In [0]:
dataset['tweet'] = dataset.tweet.apply(lambda x: ' '.join(x))

Memasukkan label ke array

In [0]:
dataset['label'] = dataset.status.map(lambda x: x)

In [0]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(dataset['tweet'])

In [15]:
print(counts[:5])

  (0, 1715)	1
  (0, 4248)	1
  (0, 37085)	1
  (0, 2486)	1
  (0, 36375)	1
  (1, 2486)	1
  (1, 36375)	1
  (1, 13185)	1
  (1, 9869)	1
  (1, 28573)	1
  (2, 1715)	1
  (2, 14328)	1
  (2, 21932)	1
  (2, 8839)	1
  (2, 466)	1
  (2, 10073)	1
  (2, 28935)	1
  (3, 18837)	1
  (3, 29840)	1
  (3, 10628)	1
  (3, 4838)	1
  (3, 10577)	1
  (3, 31068)	1
  (3, 23900)	1
  (3, 975)	1
  (3, 36000)	1
  (3, 5072)	1
  (4, 28573)	1
  (4, 466)	1
  (4, 23215)	1
  (4, 3936)	1
  (4, 37084)	1
  (4, 31856)	1


Disini mentransformasikan semua feature kata menggunakan TF IDF

In [0]:
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

In [17]:
print(counts[:5])

  (0, 37085)	0.5911705953129078
  (0, 36375)	0.2822194474686629
  (0, 4248)	0.3843499205847922
  (0, 2486)	0.47674485769290337
  (0, 1715)	0.44255958911507276
  (1, 36375)	0.2866682929106074
  (1, 28573)	0.38428108764986996
  (1, 13185)	0.49885978763514066
  (1, 9869)	0.5355187636154758
  (1, 2486)	0.4842601590165411
  (2, 28935)	0.3746459280373387
  (2, 21932)	0.2760387849741051
  (2, 14328)	0.21527389492118496
  (2, 10073)	0.7561577668564705
  (2, 8839)	0.18528478090139275
  (2, 1715)	0.2457130613228591
  (2, 466)	0.2657446914706499
  (3, 36000)	0.38496965117902543
  (3, 31068)	0.322521532751217
  (3, 29840)	0.2868751168311405
  (3, 23900)	0.17759626056082173
  (3, 18837)	0.3707445553998642
  (3, 10628)	0.2008833216432625
  (3, 10577)	0.2868751168311405
  (3, 5072)	0.34646183022969057
  (3, 4838)	0.3628687482176093
  (3, 975)	0.34946812299802377
  (4, 37084)	0.2176563413269418
  (4, 31856)	0.28790259324420076
  (4, 28573)	0.23764296132175766
  (4, 23215)	0.44937909480755067
  (4, 393

Kemudian setelah semua kata memiliki bobot tersendiri sesuai dengan transformasi vektor TF IDF, disini pembagian data training dan data testing dengan alokasi 80% train:20% test

In [0]:
feature_train, feature_test, target_train, target_test = train_test_split(counts, dataset['label'], train_size=0.8, test_size=0.2, random_state=52)

In [0]:
model = MultinomialNB()
model.fit(feature_train, target_train)
predicted = model.predict(feature_test)

Untuk menghitung akurasi model yang sudah dibuat, dapat menggunakan accuracy_score dari scikitlearn

In [0]:
accuracy = accuracy_score(target_test, predicted)

In [21]:
print(accuracy)

0.983375


Untuk pengujian, menggunakan confusion matrix sebagai berikut ini:

In [0]:
c_matrix = confusion_matrix(target_test, predicted)

In [25]:
print(c_matrix)

[[3940   73]
 [  60 3927]]


Untuk melihat laporan klasifikasi, menggunakan classification_report 

In [0]:
c_report = classification_report(target_test, predicted)

In [27]:
print(c_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4013
           1       0.98      0.98      0.98      3987

    accuracy                           0.98      8000
   macro avg       0.98      0.98      0.98      8000
weighted avg       0.98      0.98      0.98      8000

